# Import necessary libraries

In [1]:
import io
import random
import numpy as np
import re, string, unicodedata
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages
#nltk.download('punkt') # first-time use only
#nltk.download('wordnet') # first-time use only

True

# import raw text

In [4]:
f=open('chatbot.txt','r',errors = 'ignore')
raw=f.read()
raw

'Coronavirus is a large family of viruses that are known to cause illness ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS).\n\nA novel coronavirus (CoV) is a new strain of coronavirus that has not been previously identified in humans.\n\nCan humans become infected with a novel coronavirus of animal source?\nDetailed investigations found that SARS-CoV was transmitted from civet cats to humans in China in 2002 and MERS-CoV from dromedary camels to humans in Saudi Arabia in 2012. Several known coronaviruses are circulating in animals that have not yet infected humans. As surveillance improves around the world, more coronaviruses are likely to be identified.\n\nthe symptoms It depends on the virus, but common signs include respiratory symptoms, fever, cough, shortness of breath, and breathing difficulties. In more severe cases, infection can cause pneumonia, severe acute respiratory syndrome, k

# Lowercase except some special strings

In [5]:
import itertools

get list of specific name which are all uppercase

In [6]:
specific_word = [' '.join(b) for a, b in itertools.groupby(raw.split(), key=str.isupper) if a]

In [7]:
specific_word

['(MERS)', '(SARS). A', 'WHO', 'WHO', 'WHO', '(SARI),', 'SARI', 'WHO']

remove duplicated specific name in list 

In [8]:
specific_word = list(set(specific_word))
specific_word

['(SARS). A', 'SARI', 'WHO', '(MERS)', '(SARI),']

convert a string to lowercase except some special strings (specific_word)

In [9]:
toLower = lambda x: " ".join( a if a in specific_word  else a.lower()
            for a in x.split() )

In [10]:
# print (toLower(data))

In [11]:
data = toLower(raw)
data

'coronavirus is a large family of viruses that are known to cause illness ranging from the common cold to more severe diseases such as middle east respiratory syndrome (MERS) and severe acute respiratory syndrome (sars). a novel coronavirus (cov) is a new strain of coronavirus that has not been previously identified in humans. can humans become infected with a novel coronavirus of animal source? detailed investigations found that sars-cov was transmitted from civet cats to humans in china in 2002 and mers-cov from dromedary camels to humans in saudi arabia in 2012. several known coronaviruses are circulating in animals that have not yet infected humans. as surveillance improves around the world, more coronaviruses are likely to be identified. the symptoms it depends on the virus, but common signs include respiratory symptoms, fever, cough, shortness of breath, and breathing difficulties. in more severe cases, infection can cause pneumonia, severe acute respiratory syndrome, kidney fail

# Tokenisation

In [27]:
sent_tokens = nltk.sent_tokenize(data)# converts to list of sentences 
word_token = nltk.word_tokenize(data)# converts to list of words

In [13]:
print("The number of sentences is", len(sent_tokens))
print("The number of tokens is", len(word_token))

The number of sentences is 16
The number of tokens is 405


In [29]:
print(sent_tokens)

['coronavirus is a large family of viruses that are known to cause illness ranging from the common cold to more severe diseases such as middle east respiratory syndrome (MERS) and severe acute respiratory syndrome (sars).', 'a novel coronavirus (cov) is a new strain of coronavirus that has not been previously identified in humans.', 'can humans become infected with a novel coronavirus of animal source?', 'detailed investigations found that sars-cov was transmitted from civet cats to humans in china in 2002 and mers-cov from dromedary camels to humans in saudi arabia in 2012. several known coronaviruses are circulating in animals that have not yet infected humans.', 'as surveillance improves around the world, more coronaviruses are likely to be identified.', 'the symptoms it depends on the virus, but common signs include respiratory symptoms, fever, cough, shortness of breath, and breathing difficulties.', 'in more severe cases, infection can cause pneumonia, severe acute respiratory sy

In [30]:
print(word_token)

['coronavirus', 'is', 'a', 'large', 'family', 'of', 'viruses', 'that', 'are', 'known', 'to', 'cause', 'illness', 'ranging', 'from', 'the', 'common', 'cold', 'to', 'more', 'severe', 'diseases', 'such', 'as', 'middle', 'east', 'respiratory', 'syndrome', '(', 'MERS', ')', 'and', 'severe', 'acute', 'respiratory', 'syndrome', '(', 'sars', ')', '.', 'a', 'novel', 'coronavirus', '(', 'cov', ')', 'is', 'a', 'new', 'strain', 'of', 'coronavirus', 'that', 'has', 'not', 'been', 'previously', 'identified', 'in', 'humans', '.', 'can', 'humans', 'become', 'infected', 'with', 'a', 'novel', 'coronavirus', 'of', 'animal', 'source', '?', 'detailed', 'investigations', 'found', 'that', 'sars-cov', 'was', 'transmitted', 'from', 'civet', 'cats', 'to', 'humans', 'in', 'china', 'in', '2002', 'and', 'mers-cov', 'from', 'dromedary', 'camels', 'to', 'humans', 'in', 'saudi', 'arabia', 'in', '2012.', 'several', 'known', 'coronaviruses', 'are', 'circulating', 'in', 'animals', 'that', 'have', 'not', 'yet', 'infected'

# Remove stopword & punctuation

In [33]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
word_tokens = []

In [43]:
count = 0
for i in iter(stop_words):
    count = count + 1
    if count == 20:
        break
    print(i)

more
or
him
which
so
y
further
doesn't
after
only
was
that'll
couldn
during
you
were
once
yourself
has


In [16]:
for i in word_token:
    if (i not in stop_words and i not in string.punctuation):
        word_tokens.append(i)

In [17]:
print("The number of total tokens after removing stopwords are", len((word_tokens)))

The number of total tokens after removing stopwords are 220


In [18]:
print('after I removed stop words and punctuation:')
word_tokens[:10]

after I removed stop words and punctuation:


['coronavirus',
 'large',
 'family',
 'viruses',
 'known',
 'cause',
 'illness',
 'ranging',
 'common',
 'cold']

# Preprocessing

## see what happen in Lemmatization

In [19]:
lemmer = WordNetLemmatizer()
def print_lemma_stem(word_list):
    for word in word_list:
        print(f'{word} ==> {lemmer.lemmatize(word)}')

Fetch first 10 result from a list of word_tokens after lemmatizing

In [20]:
print_lemma_stem(word_tokens[:10])

coronavirus ==> coronavirus
large ==> large
family ==> family
viruses ==> virus
known ==> known
cause ==> cause
illness ==> illness
ranging ==> ranging
common ==> common
cold ==> cold


## see what happen in stemming

In [21]:
from nltk.stem.porter import PorterStemmer
# Instantiate stemming class
stemmer = PorterStemmer() 
stem_porter = []

def print_porter_stem(word_list):
    for word in word_list:
        print(f'{word} ==> {stemmer.stem(word)}')

print_porter_stem(word_tokens[:10])

coronavirus ==> coronaviru
large ==> larg
family ==> famili
viruses ==> virus
known ==> known
cause ==> caus
illness ==> ill
ranging ==> rang
common ==> common
cold ==> cold


จากการเปรียบเทียบการทำงานของ stemming และ lemmatization พบว่า stemmimg มีการเปลี่ยนแปลงที่มากกว่า แต่บางครั้งก็เปลี่ยนแปลงจนผิดความหมาย เช่นจากตัวอย่างด้านบนจะเห็นว่า cause ถูกเปลี่ยนเป็น caus ซึ่งกลายเป็นคำไม่มีความหมาย
ดังนั้น จึงเลือกทำแค่ Lemmatization เพียงอย่างเดียวก็น่าจะเพียงพอ

# Define function of Lemmatization

In [22]:
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.translate(remove_punct_dict)))

# Keyword matching

In [23]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

# Generating Response from cosine similarity

In [24]:
pip install google

In [25]:
from googlesearch import search  
def response(user_response):
    robo_response=''  
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
#     vals = linear_kernel(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        query = user_response
        links = []
        for j in search(query, tld="co.in", num=10, stop=10, pause=2): 
            links.append(j) 
        robo_response= print("I don't know "+ user_response+" \n \n But you can find more in this link",links[0])
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response


In [45]:

flag=True
print("BotK: My name is BotK. I will answer your question about Coronavirus. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("BotK: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("BotK: "+greeting(user_response))
            else:
                print("BotK: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("BotK: Bye! take care your self to pass this situation together")


BotK: My name is BotK. I will answer your question about Coronavirus. If you want to exit, type Bye!
hi
BotK: hello
What is coronavirus
BotK: a novel coronavirus (cov) is a new strain of coronavirus that has not been previously identified in humans.
treatment
BotK: there is no specific treatment for disease caused by a novel coronavirus.
symptoms
BotK: the symptoms it depends on the virus, but common signs include respiratory symptoms, fever, cough, shortness of breath, and breathing difficulties.
protection
BotK: standard recommendations protection to reduce exposure to and transmission of a range of illnesses include maintaining basic hand and respiratory hygiene, and safe food practices and avoiding close contact, when possible, with anyone showing symptoms of respiratory illness such as coughing and sneezing.
vaccine
BotK: when a disease is new, there is no vaccine until one is developed.
KMUTT
BotK: I don't know kmutt 
 
 But you can find more in this link https://www.kmutt.ac.th/